In [1]:
!nvidia-smi

Fri Jul  7 17:13:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
! pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.0 MB/s eta 0:00:00


In [3]:
import os
import csv
import requests
from tqdm import tqdm
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

In [4]:
from google.colab import drive
drive.mount('/content/drive')

! unzip -q '/content/drive/MyDrive/cleaned_img.zip' -d '/content/data' # Load Preprocessed Images of Memes

Mounted at /content/drive


In [5]:
# Load Pre-trained Image Captioning Model BLIP
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to("cuda")

In [6]:
def generate_caption(image):
  inputs = processor(image, return_tensors="pt").to("cuda")
  out = model.generate(**inputs)
  generated_text = processor.decode(out[0], skip_special_tokens=True)

  return generated_text

def write_csv(data):
    with open(caption_csv, 'a+') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(data)

In [7]:
img_dir = "/content/data"
caption_csv = "/content/BLIP_caption.csv"

# Extract captions from memes and save them in csv
for i,img_path in enumerate(tqdm(os.listdir(img_dir))):
        if img_path.endswith('.png'):
            img = os.path.join(img_dir, img_path)
            raw_image = Image.open(img)
            caption = generate_caption(raw_image)
            data = [img_path, caption]
            write_csv(data)

  0%|          | 0/12140 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 12140/12140 [1:07:22<00:00,  3.00it/s]


In [11]:
# Save the csv file in Google Drive
!cp -r /content/BLIP_caption.csv /content/drive/MyDrive